In [4]:
#pip install pyreadr

In [64]:
import pandas as pd
import pyreadr
import numpy as np
from functools import reduce

# Data Cleaning

## 0. Files for helping unify the USstate presentation

In [3]:
delineation_df = pd.read_excel('./data/delineation_file.xls')
delineation_df = delineation_df.iloc[1:1918]
delineation_df = delineation_df.rename(columns=delineation_df.iloc[0])
delineation_df = delineation_df.drop(1)
delineation_df['FIPS State Code'] = delineation_df['FIPS State Code'].astype(int)
delineation_df['CBSA Code'] = delineation_df['CBSA Code'].astype(int)
delineation_df = delineation_df[['CBSA Code', 'State Name', 'FIPS State Code', 'FIPS County Code']]

abbr_name = pd.read_csv('./data/abbr-name.csv')
abbr_name['abbr'] = abbr_name['abbr'].str.lower()

code_merged = pd.merge(delineation_df, abbr_name, left_on='State Name', right_on='full', how='left')
code_merged = code_merged.drop(columns=['full'])
code_merged.drop_duplicates()

,CBSA Code,State Name,FIPS State Code,FIPS County Code,abbr
0,10100,South Dakota,46,013,sd
1,10100,South Dakota,46,045,sd
2,10140,Washington,53,027,wa
3,10180,Texas,48,059,tx
4,10180,Texas,48,253,tx
...,...,...,...,...,...
1911,49700,California,6,101,ca
1912,49700,California,6,115,ca
1913,49740,Arizona,4,027,az
1914,49780,Ohio,39,119,oh


In [4]:
state = abbr_name['abbr'].to_list()

## 1. Crime Data 2011-2020

In [129]:
crime_2011 = pd.read_csv('./data/crime/crime_2011.csv')
crime_2012 = pd.read_csv('./data/crime/crime_2012.csv')
crime_2013 = pd.read_csv('./data/crime/crime_2013.csv')
crime_2014 = pd.read_csv('./data/crime/crime_2014.csv')
crime_2015 = pd.read_csv('./data/crime/crime_2015.csv')
crime_2016 = pd.read_csv('./data/crime/crime_2016.csv')
crime_2017 = pd.read_csv('./data/crime/crime_2017.csv')
crime_2018 = pd.read_csv('./data/crime/crime_2018.csv')
crime_2019 = pd.read_csv('./data/crime/crime_2019.csv')
crime_2020 = pd.read_csv('./data/crime/crime_2020.csv')

In [130]:
crime_11_20 = pd.concat([crime_2011, crime_2012, crime_2013, crime_2014, crime_2015, crime_2016, crime_2017, 
                         crime_2018, crime_2019, crime_2020])
crime_11_20 = crime_11_20.drop("Unnamed: 0",axis=1)
max_pop = crime_11_20['state_abb'][crime_11_20['population'].idxmax()]
print(max_pop)
min_pop = crime_11_20['state_abb'][crime_11_20['population'].idxmin()]
print(min_pop)

165120    NY
165120    NY
165120    NY
165120    NY
165120    NY
165120    NY
165120    NY
165120    NJ
165120    NJ
165120    NJ
Name: state_abb, dtype: object
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
264    AK
Name: state_abb, dtype: object


In [131]:
crime_11_20 = crime_11_20.groupby(['state_abb','year','month']).sum().reset_index()
crime_11_20['state_abb'] = crime_11_20['state_abb'].str.lower()
months = {'january': 1, 'february': 2, 'march': 3, 'april':4, 'may':5, 'june':6, 'july':7, 'august':8, 
          'september':9, 'october':10, 'november':11, 'december':12}
for index, row in crime_11_20.iterrows():
    crime_11_20['month'][index] = months[row['month']]
crime_11_20 = crime_11_20.rename(columns={'state_abb': 'USstate'})
crime_11_20.to_csv('./data/crime/crime.csv')
crime_11_20

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_58094/852850518.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_11_20['month'][index] = months[row['month']]


,USstate,year,month,population,actual_rape_total
0,ak,2011,4,725910,26
1,ak,2011,8,725910,45
2,ak,2011,12,725910,32
3,ak,2011,2,725910,29
4,ak,2011,1,725910,36
...,...,...,...,...,...
6475,wy,2020,3,593429,29
6476,wy,2020,5,593429,18
6477,wy,2020,11,593429,23
6478,wy,2020,10,593429,40


## 2. Alcohol Data for 2012-2020

In [117]:
alcohol_12 = pd.read_csv('./data/alcohol/CBP2012.csv')
alcohol_13 = pd.read_csv('./data/alcohol/CBP2013.csv')
alcohol_14 = pd.read_csv('./data/alcohol/CBP2014.csv')
alcohol_15 = pd.read_csv('./data/alcohol/CBP2015.csv')
alcohol_16 = pd.read_csv('./data/alcohol/CBP2016.csv')
alcohol_17 = pd.read_csv('./data/alcohol/CBP2017.csv')
alcohol_18 = pd.read_csv('./data/alcohol/CBP2018.csv')
alcohol_19 = pd.read_csv('./data/alcohol/CBP2019.csv')
alcohol_20 = pd.read_csv('./data/alcohol/CBP2020.csv')
alcohol_12_20 = pd.concat([alcohol_12, alcohol_13, alcohol_14, alcohol_15, alcohol_16, 
                           alcohol_17, alcohol_18, alcohol_19, alcohol_20])
alcohol_12_20 = pd.merge(alcohol_12_20, abbr_name, left_on='Geographic Area Name (NAME)', right_on='full', how='left')
alcohol_12_20 = alcohol_12_20.dropna(subset=['full'])
alcohol_12_20 = alcohol_12_20.drop(columns=['full', 'Geographic Area Name (NAME)'])
alcohol_12_20 = alcohol_12_20.rename(columns={'Year (YEAR)': 'year','abbr': 'USstate'})
alcohol_df = alcohol_12_20.groupby(['year','USstate']).sum().reset_index()
alcohol_state = set(alcohol_df['USstate'].to_list())
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
full_alcohol = pd.DataFrame(columns=['USstate', 'year', 'month', 'factor'])
for i in range(len(alcohol_state)):
    for j in year:
        for k in month:
            
            if k == 12:
                full_alcohol = full_alcohol.append({'USstate': state[i], 'year': j, 'month': k, 'factor': 1}, 
                                                   ignore_index=True)
            else:
                full_alcohol = full_alcohol.append({'USstate': state[i], 'year': j, 'month': k, 'factor': 0}, 
                                                   ignore_index=True)
print(len(alcohol_state))

50


In [118]:
alcohol_df

,year,USstate,Number of establishments (ESTAB)
0,2012,ak,101.0
1,2012,al,367.0
2,2012,ar,318.0
3,2012,az,208.0
4,2012,ca,3733.0
...,...,...,...
445,2020,vt,116.0
446,2020,wa,259.0
447,2020,wi,401.0
448,2020,wv,70.0


In [108]:
alcohol = pd.merge(alcohol_df, full_alcohol, on=['USstate', 'year'], how='right')
alcohol['Number of establishments (ESTAB)'] = alcohol['Number of establishments (ESTAB)'] * alcohol['factor']
alcohol = alcohol.drop(columns=['factor'])
alcohol['Number of establishments (ESTAB)'] = alcohol['Number of establishments (ESTAB)'].replace({0:np.nan})
alcohol['Number of establishments (ESTAB)'] = alcohol['Number of establishments (ESTAB)'].interpolate(method ='cubic', limit_direction ='backward')
alcohol['Number of establishments (ESTAB)'] = alcohol['Number of establishments (ESTAB)'].interpolate(method ='linear', limit_direction ='backward')
alcohol['year'] = pd.to_numeric(alcohol['year'])
alcohol['month'] = pd.to_numeric(alcohol['month'])
alcohol.to_csv('./data/alcohol/alcohol.csv')
alcohol

,year,USstate,Number of establishments (ESTAB),month
0,2012.0,al,367.000000,1
1,2012.0,al,367.000000,2
2,2012.0,al,367.000000,3
3,2012.0,al,367.000000,4
4,2012.0,al,367.000000,5
...,...,...,...,...
5395,2020.0,wi,394.990058,8
5396,2020.0,wi,395.674004,9
5397,2020.0,wi,396.875109,10
5398,2020.0,wi,398.636174,11


## 3. Twitter Data 2012-2021

In [132]:
# use proportion of misogyny tweets
all_tweets = pd.read_excel('./data/tweet/All_Tweets_State_Month_Year_05082022.xlsx')
misogyny_tweets = pd.read_excel('./data/tweet/Misogyny_Tweets_State_Month_Year_05082022.xlsx')
tweets = pd.merge(all_tweets, misogyny_tweets, on = ['USstate', 'year', 'month.keyword: Descending'])
tweets = tweets.sort_values(by=['year', 'month.keyword: Descending'], ascending=True)
tweets = tweets.rename(columns={'month.keyword: Descending': 'month'})

tweets_state = set(tweets['USstate'].to_list())
tweets = pd.merge(tweets, full_tweet, on=['USstate', 'year', 'month'], how='right')
tweets

,USstate,year,month,nTweets_x,nTweets_y
0,al,2012,1,NaN,NaN
1,al,2012.0,2.0,51744.0,3.0
2,al,2012.0,3.0,55089.0,2.0
3,al,2012.0,4.0,38471.0,1.0
4,al,2012.0,5.0,36891.0,1.0
...,...,...,...,...,...
5635,va,2021.0,8.0,31318.0,1.0
5636,va,2021,9,NaN,NaN
5637,va,2021,10,NaN,NaN
5638,va,2021,11,NaN,NaN


In [133]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
full_tweet = pd.DataFrame(columns=['USstate', 'year', 'month'])
for i in range(len(tweets_state)):
    for j in year:
        for k in month:
            full_tweet = full_tweet.append({'USstate': state[i], 'year': j, 'month': k}, ignore_index=True)
print(len(tweets_state))

47


In [134]:
# use spline to fill the missing data # cubic
tweets = pd.merge(tweets, full_tweet, on=['USstate', 'year', 'month'], how='right')
tweets['nTweets_x'] = tweets['nTweets_x'].interpolate(method ='linear', limit_direction ='backward')
tweets['nTweets_y'] = tweets['nTweets_y'].interpolate(method ='linear', limit_direction ='backward')
tweets['year'] = pd.to_numeric(tweets['year'])
tweets['month'] = pd.to_numeric(tweets['month'])
tweets.to_csv('./data/tweet/tweets_df.csv')
tweets

,USstate,year,month,nTweets_x,nTweets_y
0,al,2012,1,51744.00,3.0
1,al,2012,2,51744.00,3.0
2,al,2012,3,55089.00,2.0
3,al,2012,4,38471.00,1.0
4,al,2012,5,36891.00,1.0
...,...,...,...,...,...
5635,va,2021,8,31318.00,1.0
5636,va,2021,9,31468.75,1.0
5637,va,2021,10,31619.50,1.0
5638,va,2021,11,31770.25,1.0


# 4. Merge all datasets

In [135]:
# alcohol and twitter
al_twitter = pd.merge(tweets, alcohol, right_on=['USstate','year','month'], left_on=['USstate', 'year','month'], how='left')
al_twitter = pd.merge(al_twitter, abbr_name, left_on='USstate', right_on='abbr', how='left')
al_twitter = al_twitter.dropna()

# add crime
merged_df = pd.merge(al_twitter, crime_11_20, left_on=['USstate', 'year', 'month'], right_on=['USstate', 'year', 'month'], how='left')
merged_df['actual_rape_total'] = merged_df['actual_rape_total'].interpolate(method ='cubic', limit_direction ='backward')
merged_df

,USstate,year,month,nTweets_x,nTweets_y,Number of establishments (ESTAB),abbr,full,population,actual_rape_total
0,al,2012,1,51744.00,3.0,367.000000,al,Alabama,4834334,106
1,al,2012,2,51744.00,3.0,367.000000,al,Alabama,4834334,70
2,al,2012,3,55089.00,2.0,367.000000,al,Alabama,4834334,78
3,al,2012,4,38471.00,1.0,367.000000,al,Alabama,4834334,86
4,al,2012,5,36891.00,1.0,367.000000,al,Alabama,4834334,101
...,...,...,...,...,...,...,...,...,...,...
5071,va,2020,8,85152.00,2.0,480.577809,va,Virginia,8600924,236
5072,va,2020,9,86418.50,1.5,480.179433,va,Virginia,8600924,193
5073,va,2020,10,87685.00,1.0,479.779695,va,Virginia,8600924,198
5074,va,2020,11,74017.25,1.0,479.384563,va,Virginia,8600924,166


In [136]:
merged_df.to_csv('./data/merged_data.csv')